# Recomendation system
## Content based filtering

In [3]:
import pandas as pd
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split

In [4]:
ratings = pd.read_csv('data/ratings.csv', low_memory=False)
ratings = ratings.drop('timestamp', axis=1)
ratings.head()

,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0
3,1,1221,5.0
4,1,1246,5.0


In [5]:
movie_metadata = pd.read_csv("data/movies_metadata.csv", low_memory=False)
movie_metadata = movie_metadata[['id', 'title']]
movie_metadata =movie_metadata[pd.to_numeric(movie_metadata['id'], errors='coerce').notnull()]
movie_metadata=movie_metadata.rename(columns = {'id':'movieId'})
movie_metadata['movieId'] = movie_metadata['movieId'].astype(int)
movie_metadata.head()

,movieId,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men
3,31357,Waiting to Exhale
4,11862,Father of the Bride Part II


In [6]:
movie_data = ratings.merge(movie_metadata, how='left', on='movieId')
movie_data.head()

,userId,movieId,rating,title
0,1,110,1.0,Three Colors: Red
1,1,147,4.5,The 400 Blows
2,1,858,5.0,Sleepless in Seattle
3,1,1221,5.0,NaN
4,1,1246,5.0,Rocky Balboa


In [7]:
df_movies_cnt = pd.DataFrame(ratings.groupby('movieId').size(), columns=['count'])
popular_movies = list(set(df_movies_cnt.query('count >= 150').index))
movies_filter = ratings.movieId.isin(popular_movies).values

df_users_cnt = pd.DataFrame(ratings.groupby('userId').size(),columns=['count'])
active_users = list(set(df_users_cnt.query('count >= 15').index))
users_filter = ratings.userId.isin(active_users).values

df_ratings_filtered = ratings[movies_filter & users_filter]

In [8]:
reader = Reader()
ratings_by_users = Dataset.load_from_df(df_ratings_filtered[['userId', 'movieId', 'rating']], reader)
train_df, test_df = train_test_split(ratings_by_users, test_size=.2)

In [10]:
svd_model = SVD()
svd_model_trained = svd_model.fit(train_df)

In [23]:
def get_recommendations(user_id):
    pred_series= []
    users_ratings = ratings[ratings['userId'] == user_id]
    for movie_id, name in zip(movie_metadata.index, movie_metadata['title']):
        rating_real = ratings.query(f'movieId == {movie_id}')['rating'].values[0] if movie_id in users_ratings['movieId'].values else 0
        rating_pred = svd_model_trained.predict(user_id, movie_id, rating_real, verbose=False)
        pred_series.append([movie_id, name, rating_pred.est, rating_real])

    # print the results
    df_recommendations = pd.DataFrame(pred_series, columns=['movieId', 'title', 'predicted_rating', 'actual_rating'])
    return df_recommendations.sort_values(by='predicted_rating', ascending=False).head(10)

In [24]:
get_recommendations(400)

,movieId,title,predicted_rating,actual_rating
260,260,L'Enfer,4.691972,4.0
527,527,Serial Mom,4.623845,4.0
912,912,The Reluctant Debutante,4.619116,4.0
1207,1207,Chinatown,4.587651,0.0
17,17,Four Rooms,4.537963,0.0
1196,1196,Braindead,4.535036,4.0
7669,7669,The Way Ahead,4.526326,0.0
62,62,Don't Be a Menace to South Central While Drink...,4.514157,2.0
4427,4427,Torch Song Trilogy,4.476415,0.0
1949,1949,The Negotiator,4.465953,0.0
